In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPool2D
from keras.utils import to_categorical,np_utils
from keras import backend as K
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [2]:
train = pd.read_csv("final_train.csv")
test = pd.read_csv("final_test.csv")

In [3]:
train.head()

,Unnamed: 0,ID,0,1,2,3,4,5,6,7,...,775,776,777,778,779,780,781,782,783,Class
0,0,0.jpg,82,220,197,168,162,162,198,191,...,49,52,59,73,95,122,145,172,180,MIDDLE
1,1,1.jpg,27,34,60,73,77,89,102,100,...,57,50,43,43,43,43,50,51,58,OLD
2,2,100.jpg,37,22,18,15,24,35,32,42,...,49,46,39,40,36,13,40,54,122,MIDDLE
3,3,10000.jpg,15,21,38,36,42,45,40,38,...,89,96,92,85,78,75,70,51,34,MIDDLE
4,4,10001.jpg,84,75,78,80,80,91,104,123,...,92,82,92,101,98,101,98,133,145,YOUNG


In [4]:
test.head()

,Unnamed: 0,ID,0,1,2,3,4,5,6,7,...,774,775,776,777,778,779,780,781,782,783
0,0,10.jpg,12,11,12,15,13,10,14,45,...,106,108,116,136,141,118,98,94,89,78
1,1,1000.jpg,53,38,25,25,35,80,127,123,...,31,30,29,28,27,23,24,22,26,29
2,2,10002.jpg,10,7,5,7,5,3,7,0,...,162,160,158,159,153,150,147,145,142,140
3,3,10003.jpg,24,35,25,13,31,58,79,105,...,69,44,27,45,83,109,103,74,65,60
4,4,10005.jpg,95,94,80,100,108,134,141,160,...,147,134,132,145,155,180,169,122,113,110


In [5]:
testID = test["ID"]

In [6]:
train.drop(["Unnamed: 0","ID"],axis=1,inplace=True)
test.drop(["Unnamed: 0","ID"],axis=1,inplace=True)

In [7]:
x_train = np.array(train.iloc[:,0:784])
y_train = np.array(train.iloc[:,784])
x_test = np.array(test)

In [8]:
img_rows = 28
img_cols = 28
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [9]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (19906, 28, 28, 1)
19906 train samples
6636 test samples


In [10]:
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)


y_train = to_categorical(y_train,num_classes=3)

In [11]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),activation="relu",input_shape=input_shape))
model.add(Conv2D(64,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(32,kernel_size=(3,3),activation="relu"))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(3,activation="softmax"))

model.compile(loss = keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Adam(),
             metrics = ["accuracy"])

model.fit(x_train,y_train,batch_size = 100,
         epochs = 15,
         validation_split = 0.1)



Train on 17915 samples, validate on 1991 samples
Epoch 1/15
17915/17915 [==============================] - 77s - loss: 0.8841 - acc: 0.5783 - val_loss: 0.8064 - val_acc: 0.6344
Epoch 2/15
17915/17915 [==============================] - 79s - loss: 0.8067 - acc: 0.6323 - val_loss: 0.7821 - val_acc: 0.6339
Epoch 3/15
17915/17915 [==============================] - 84s - loss: 0.7673 - acc: 0.6557 - val_loss: 0.7347 - val_acc: 0.6645
Epoch 4/15
17915/17915 [==============================] - 79s - loss: 0.7316 - acc: 0.6752 - val_loss: 0.7265 - val_acc: 0.6770
Epoch 5/15
17915/17915 [==============================] - 82s - loss: 0.7035 - acc: 0.6939 - val_loss: 0.6973 - val_acc: 0.6851
Epoch 6/15
17915/17915 [==============================] - 82s - loss: 0.6738 - acc: 0.7071 - val_loss: 0.6669 - val_acc: 0.6976
Epoch 7/15
17915/17915 [==============================] - 85s - loss: 0.6486 - acc: 0.7200 - val_loss: 0.6623 - val_acc: 0.7172
Epoch 8/15
17915/17915 [==============================]

In [12]:
preds = model.predict(x_test)

In [13]:
preds_label = []
for i in range(len(preds)):
    preds_label.append(np.argmax(preds[i]))

In [14]:
classes = le.inverse_transform(preds_label)


In [15]:
#submission
subdf = pd.DataFrame({"Class" : classes,"ID" : testID})

In [17]:
#write to csv
subdf.to_csv("cnn_submission2.csv",index = False)